<a href="https://colab.research.google.com/github/brayanricardo13/Analisis-Econometrico/blob/main/VAR_Value_at_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

In [ ]:
pip install PyPortfolioOpt

# **VAR - Value at risk**

In [ ]:
import yfinance as yf ,numpy as np, datetime as dt, matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
from scipy.stats import norm

In [ ]:
tickers = ["MSFT","AAPL","AMZN"]
weight = np.array([1/3,1/3,1/3])

In [ ]:
Start = dt.datetime(2019,1,1)
end = dt.datetime.now()

df = yf.download(tickers,Start,end,interval="1wk")["Adj Close"]
df=df.dropna()
df

In [ ]:
df.describe()

In [ ]:
df.plot()
plt.show()

In [ ]:
retourns = df.pct_change() # Retornos 
retourns.plot()
retourns

In [ ]:
cov_matrix = retourns.cov() # Matriz de varianzas y Covarianzas 
cov_matrix 

In [ ]:
corr = retourns.corr() # correlaciones 
corr

In [ ]:
avg_retorns = retourns.mean() # Promedio de retornos 
avg_retorns

In [ ]:
count = retourns.count() # Numero de retornos 
count

In [ ]:
Port_mean =avg_retorns @ weight
Port_std = np.sqrt(weight.T @cov_matrix @weight  )

In [ ]:
Port_mean # Media del Portafolio 

In [ ]:
Port_std # Desviacion estandar del portafolio 

In [ ]:
x = np.arange(-0.05,0.05,0.001)
norm_dist = norm.pdf(x,Port_mean,Port_std)
plt.plot(x,norm_dist,color="r")
plt.show()

In [ ]:
confidence_level = 0.05
VaR = norm.ppf(confidence_level,Port_mean,Port_std)
VaR

In [ ]:
num_days = 5 
VaR*np.sqrt(num_days)

# **Portafolio Eficiente**

In [ ]:
acoes = ["MSFT","AAPL","AMZN"]
carteira = pd.DataFrame()
for i in acoes:
  carteira[i] = yf.download(i,Start,end,interval="1wk")['Adj Close']
carteira=carteira.dropna()
carteira

In [ ]:
sns.set()
carteira_normalizada = carteira/carteira.iloc[0]
carteira_normalizada.plot(figsize=(21,9))

In [ ]:
retorno = carteira.pct_change()
retorno

In [ ]:
retorno_anual = retorno.mean() * 264
retorno_anual

In [ ]:
cov = retorno.cov() * 264
cov

In [ ]:
corr = retorno.corr()
corr

In [300]:
num_acoes = len(acoes)
num_carteiras = 10000

lista_retornos = []
lista_volatilidade = []
lista_pesos = []
lista_sharpe_ratio = []

In [301]:
for carteira in range(num_carteiras):
  peso = np.random.random(num_acoes)
  peso /= np.sum(peso)
  lista_pesos.append(peso)
  retorno_esperado = np.dot(peso, retorno_anual)
  lista_retornos.append(retorno_esperado)
  volatilidade = np.sqrt(np.dot(peso.T, np.dot(cov, peso)))
  lista_volatilidade.append(volatilidade)
  sharpe_ratio = retorno_esperado/volatilidade
  lista_sharpe_ratio.append(sharpe_ratio)

In [ ]:
dic_carteiras = {'Retorno': lista_retornos, 'Volatilidade': lista_volatilidade, 'Sharpe Ratio': lista_sharpe_ratio}
for contar,acao in enumerate(acoes):
  dic_carteiras[acao+' Peso'] = [Peso[contar] for Peso in lista_pesos]
portfolios = pd.DataFrame(dic_carteiras)
colunas = ['Retorno', 'Volatilidade', 'Sharpe Ratio'] + [acao+' Peso' for acao in acoes]

portfolios = portfolios[colunas]
portfolios

In [ ]:
plt.style.use('seaborn-dark')
portfolios.plot.scatter(x='Volatilidade', y='Retorno', c='Sharpe Ratio', edgecolors='black', figsize=(21,9), grid=True)
plt.xlabel('Volatilidade', fontsize=16)
plt.ylabel('Retorno Esperado', fontsize=16)
plt.title('Fronteira Eficiente de Markowitz', fontsize=22)
plt.savefig('Fronteira.png', dpi = 500)

In [ ]:
sns.scatterplot(x = 'Volatilidade', y = 'Retorno', hue = 'Sharpe Ratio', data = portfolios, legend=None)
sns.set_xticklabels('Volatilidade', fontsize = 16)
sns.set_yticklabels('Retorno Esperado', fontsize = 16)
sns.set_title('Fronteira Eficiente de Markowitz', fontsize = 22)

In [305]:
menor_volatilidade = portfolios['Volatilidade'].min()
carteira_min_variancia = portfolios.loc[portfolios['Volatilidade'] == menor_volatilidade]
carteira_min_variancia.T

,9962
Retorno,1.853918
Volatilidade,0.571505
Sharpe Ratio,3.243923
MSFT Peso,0.552292
AAPL Peso,0.134145
AMZN Peso,0.313564


In [ ]:
maior_sharpe = portfolios['Sharpe Ratio'].max()
carteira_maior_sharpe = portfolios.loc[portfolios['Sharpe Ratio'] == maior_sharpe]
carteira_maior_sharpe.T